In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# for PyCharm
import os

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [3]:
# !dvc pull data/merged/metadata

In [3]:
df = pd.read_csv('data/merged/metadata/dataset.csv')
df.head()

,title_merge,brand,collection_name,color,images_path,price,pricecurrency,url,website
0,01 low,Autry,"['category-women', 'category-men']",White,['data/raw_new/images/sneakerbaas/category-wom...,129.99,EUR,['https://www.sneakerbaas.com/collections/snea...,"['sneakerbaas', 'sneakerbaas']"
1,01 low m,Autry,"['category-men', 'category-men', 'category-men']",BLACK,['data/raw_new/images/sneakerbaas/category-men...,149.99,EUR,['https://www.sneakerbaas.com/collections/snea...,"['sneakerbaas', 'sneakerbaas', 'sneakerbaas']"
2,01 low man,Autry,['category-men'],LEAT/LEAT RED,['data/raw_new/images/sneakerbaas/category-men...,149.99,EUR,['https://www.sneakerbaas.com/collections/snea...,['sneakerbaas']
3,01 low w,Autry,"['category-women', 'category-women', 'category...",BI POW,['data/raw_new/images/sneakerbaas/category-wom...,149.99,EUR,['https://www.sneakerbaas.com/collections/snea...,"['sneakerbaas', 'sneakerbaas', 'sneakerbaas', ..."
4,01 low wom,Autry,['category-women'],WHT/PETROL,['data/raw_new/images/sneakerbaas/category-wom...,149.99,EUR,['https://www.sneakerbaas.com/collections/snea...,['sneakerbaas']


## Preparations

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4944 entries, 0 to 4943
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_merge      4944 non-null   object 
 1   brand            4944 non-null   object 
 2   collection_name  4944 non-null   object 
 3   color            4631 non-null   object 
 4   images_path      4944 non-null   object 
 5   price            4944 non-null   float64
 6   pricecurrency    4944 non-null   object 
 7   url              4944 non-null   object 
 8   website          4944 non-null   object 
dtypes: float64(1), object(8)
memory usage: 347.8+ KB


In [5]:
df.describe(include='all')

,title_merge,brand,collection_name,color,images_path,price,pricecurrency,url,website
count,4944,4944,4944,4631,4944,4944.000000,4944,4944,4944
unique,4944,106,436,3284,4943,NaN,3,4944,268
top,01 low,Nike,[nan],['black'],"[[nan, nan, nan]]",NaN,USD,['https://www.sneakerbaas.com/collections/snea...,['kickscrew']
freq,1,1215,1830,141,2,NaN,2444,1,1830
mean,NaN,NaN,NaN,NaN,NaN,1070.301475,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,3129.078282,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,11.950000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,80.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,115.485000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,170.000000,NaN,NaN,NaN


In [6]:
unique_brands = df.brand.value_counts()
unique_brands

brand
Nike          1215
adidas         375
Converse       354
Air Jordan     277
Puma           255
              ... 
ETONIC           1
VANS             1
Fjällräven       1
Hi-tec           1
Footshop         1
Name: count, Length: 106, dtype: int64

Уникальность представлена нечестно из-за представлений некоторых значений в виде списка-строки. Исправим ситуацию для анализа:

In [7]:
def unique_in_stringlist(obj: [str|float]) -> set[str]:
    if type(obj) is str:
        return set([item.strip('\'') for item in obj.strip('][').split(', ')])
    return obj


assert (unique_in_stringlist("['footshop', 'footshop', 'footshop', 'kickscrew']") == {'footshop', 'kickscrew'})

In [9]:
df.website.apply(unique_in_stringlist).value_counts()

website
{kickscrew}                                       2444
{footshop}                                        1548
{superkicks}                                       324
{sneakerbaas}                                      312
{kickscrew, footshop}                               65
{sneakerbaas, footshop}                             61
{footshop, superkicks}                              49
{kickscrew, superkicks}                             42
{kickscrew, footshop, superkicks}                   34
{kickscrew, sneakerbaas, footshop, superkicks}      16
{sneakerbaas, kickscrew, footshop}                  15
{sneakerbaas, footshop, superkicks}                 12
{sneakerbaas, kickscrew}                            11
{sneakerbaas, superkicks}                            9
{sneakerbaas, kickscrew, superkicks}                 2
Name: count, dtype: int64

In [13]:
df.color.astype(str).apply(lambda x: x.lower()).apply(unique_in_stringlist).value_counts()

color
{black}                         315
{nan}                           313
{white}                         110
{black white}                    61
{white black}                    48
                               ... 
{active red}                      1
{cool grey infrared}              1
{yellow bear}                     1
{black glow volt}                 1
{savanna, sand, cloud white}      1
Name: count, Length: 3152, dtype: int64

In [14]:
print("Emoty colors:", df[df.color.isna()].count())

Emoty colors: title_merge        313
brand              313
collection_name    313
color                0
images_path        313
price              313
pricecurrency      313
url                313
website            313
dtype: int64


In [56]:
# No color by brand
df[df.color.isna()].brand.value_counts() 

brand
Nike                  54
Asics                 38
Vans                  31
adidas                28
Converse              20
Puma                  15
Onitsuka Tiger        15
New Balance           14
Reebok                11
Skechers               8
PUMA Sportstyle        8
Puma Sportstyle        8
Air Jordan             8
Li-Ning                6
Mizuno                 5
UGG                    4
Saucony                4
adidas Originals       4
SALOMON                4
Fila                   3
Reebok Classics        3
Adidas Originals       3
Adidas                 2
Under Armour           2
Hi-Tec                 2
Crocs                  2
OFF-WHITE              2
THE NORTH FACE         1
HOKA ONE ONE           1
ANTA                   1
Dr. Martens            1
GUCCI                  1
CASIO                  1
Clarks Originals       1
FILA                   1
Nike Skateboarding     1
Name: count, dtype: int64

In [15]:
df[df.color.isna()].website.apply(unique_in_stringlist).value_counts()

website
{kickscrew}                  237
{sneakerbaas}                 35
{superkicks}                  34
{kickscrew, superkicks}        3
{sneakerbaas, footshop}        3
{sneakerbaas, superkicks}      1
Name: count, dtype: int64

Цвета довольно рандомные и при этом имеют пропуски по очевидным причинам - некоторые кроссовки не получили свой цвет при: 1) парсинге 2) удалении цвета из названия. В примере с kickscrew причиной является 1 вариант, так как цвет можно найти на веб-версии, но нет в базе данных (algolia).

In [16]:
df[df.color.isna()].url.iloc[0][2:-2]

'https://www.kickscrew.com/products/puma-1948-mid-359169-08'